Begin with importing the required libraries

In [2]:
import pickle
import pandas as pd
import numpy as np
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, r2_score
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV, LassoCV, lasso_path
from sklearn.pipeline import Pipeline 
from sklearn import linear_model

The original file may or may not be available however below is the code for reading the original file and building a dictionary with only the S. pneumoniae organism

In [3]:
# Read the data into python (The path will change depending on the location of the file)
Data = pd.read_csv("/Users/keenananderson-fears/Desktop/Dr. Liu Lab/Current AMR Project/NCBI_Python_Readable.csv", 
                  sep='\t',
                  header=None)

# First I create an empty list for the names of each object, an empty dataframe for the data to be stored and an 
# empty dictionary to append everything to using the names as my keys
List_of_Names = []
NCBI_Data = {}

# I next create the outer loop for generating all of the bacteria/antibiotic combinations from each row of my 
# original matrix
for row in Data.iterrows():
    # I then begin by selecting a single row from the original matrix to parse through with my loop, turn it into a 
    # dataframe with a single column and remove the NaN values from the dataset
    Data4 = Data.iloc[row[0],]
    Data5 = pd.DataFrame(Data4)
    Data6= Data5.dropna()
    # Next I set up some dummy variables, lists & data frames, which I can append the data to from each element of 
    # my original matrix
    DataFrame = pd.DataFrame()
    Index = []
    Column_List = pd.DataFrame()
    Data_List = []
    # Next I set my loop up to go through the data by row
    for i in Data6.iterrows():
        # I next have to get the data into dataframes which XGBoost is capable of handling
        str = ','.join(i[1]) # Since it is a string I combine all strings at all commas
        # I then split them and compute the begining/end of the features set and set this to an array, removing the 
        # third element which corresponds to the individual code
        Column_List = np.delete(np.array(str.split(',')[int(len(str.split(','))/2):]), 2)
        # I then split them and compute the begining/end of the feature values set and set this to an array, 
        #removing the third element which corresponds to the individual code
        Data_List.append([np.delete(np.array(str.split(',')[0:int(len(str.split(','))/2)]), 2)])
        # Next I isolate those elements which correspond to the individual codes and save those as my Index values
        I = str.split(',')[2]
        I = I.replace("(","").replace(")","").replace(" ","").replace(",","")
        Index.append([I])
        #Index.append([str.replace("(","").replace(")","").replace(" ","").replace(",","").split(',')[2]])
    # Once we have all our information stored we can move back to the outer loop to begin saving the information
    # into their respective dataframe objects
    Data6.columns = ["All"] # I have to rename the column list so as to call it later when making the dataframe
    Name = Data6["All"].iloc[0].split(',')[0] # I want the output of this command to be the name of my dataframe 
                                              # below
    Name = Name.replace(" ", "_").replace(":","_") # In order to the save the bacteria/antibiotic combinations as
                                                   # a file name I need to replace the spaces and colons with "_"
    # I convert my Index and Data to a dataframe to make it readable by the index parameter within the DataFrame 
    # function    
    Index = pd.DataFrame(Index)
    Data_List = pd.DataFrame(Data_List)
    Data_List.columns = ["All"]
    List_of_Names.append(Name)
    # Lastly I set up the DataFrame and save this to my dictionary with the bacteria/antibiotic name as the key 
    # using only the Streptococcus_pneumoniae organism
    DataFrame = pd.DataFrame(Data_List['All'].to_list(), columns = Column_List, index = Index)
    if Name == "Streptococcus_pneumoniae_cefuroxime":
        NCBI_Data[Name] = DataFrame
    elif Name == "Streptococcus_pneumoniae_clindamycin":
        NCBI_Data[Name] = DataFrame
    elif Name == "Streptococcus_pneumoniae_erythromycin":
        NCBI_Data[Name] = DataFrame
    elif Name == "Streptococcus_pneumoniae_meropenem":
        NCBI_Data[Name] = DataFrame
    elif Name == "Streptococcus_pneumoniae_tetracycline":
        NCBI_Data[Name] = DataFrame
    elif Name == "Streptococcus_pneumoniae_trimethoprim-sulfamethoxazole":
        NCBI_Data[Name] = DataFrame
    #NCBI_Data[Name] = DataFrame

Next we create a pickle file of the dictionary to store in our working directory

In [4]:
# Store data (serialize)
with open('NCBI_Data.pickle', 'wb') as handle:
    pickle.dump(NCBI_Data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
# Load data (deserialize)
with open('NCBI_Data.pickle', 'rb') as handle:
    NCBI_Data = pickle.load(handle)

We then concatenate our various dataframes within our dictionary based upon the union of all columns 

In [5]:
#Dataframes for each resistance for Strep
df = pd.DataFrame()
cfrx = NCBI_Data['Streptococcus_pneumoniae_cefuroxime']
cnm = NCBI_Data['Streptococcus_pneumoniae_clindamycin']
e = NCBI_Data['Streptococcus_pneumoniae_erythromycin']
m = NCBI_Data['Streptococcus_pneumoniae_meropenem']
t = NCBI_Data['Streptococcus_pneumoniae_tetracycline']
ts = NCBI_Data['Streptococcus_pneumoniae_trimethoprim-sulfamethoxazole']
 
#Concatenate from above dataframes
df = pd.concat([cfrx,cnm,e,m,t,ts], axis=0, ignore_index=True, sort=True)
 
#Remove columns with NAs and call df 'shared_variants'
shared_variants = df.dropna(axis=1, how='any', thresh=None, subset=None, inplace=False)

Our dataframe now has a column called Names with the organism and antibiotic treatment. This being a catagorical vector cannot be read directly by any statistical software and must thus be encoded. Therefore we use a on-hot-encoding method to avoid any ordinal covariates within our data.

In [6]:
data = shared_variants.iloc[:,-1:]
values = array(data)
values = values.flatten()
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
OHEData = pd.DataFrame(data=onehot_encoded, columns=["Streptococcus_pneumoniae_cefuroxime",
                                                    "Streptococcus_pneumoniae_clindamycin",
                                                    "Streptococcus_pneumoniae_erythromycin",
                                                    "Streptococcus_pneumoniae_meropenem",
                                                    "Streptococcus_pneumoniae_tetracycline",
                                                    "Streptococcus_pneumoniae_trimethoprim-sulfamethoxazole"])
print(onehot_encoded)

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]


We then take our encoded features, add them to our dataframe and remove the name column

In [6]:
test = shared_variants.iloc[:,:-1]
Final_Data = pd.concat([test, OHEData], axis=1).reindex(shared_variants.index)
Final_Data.to_csv('CBIOS_Full_Data.csv')

Next we split our data into a testing set and training set for screening and further feature selection. Using the N/log10(N) cutoff for feature screening, where N is the number of columns, we find that our ideal number of features is around 344. Following this we can use a lasso path to identify the top 344 features in terms of their effect on our Y's, AMR, variance. Feature screening is performed using glmnet in R (see CBIOS_Project.Rmd)